# Getting available actions
Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
but only one discussed here. 
* ~~web interaction that creates jobs~~
* **azure functions that resolve actions**

In [1]:
import sys
import numpy as np
import pandas as pd


sys.path.append("../..")
sys.path.append("..")
import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()


from app.connectors import cmdb_graph
from app.objects import time as t


c = cmdb_graph.CosmosdbClient()


executing local windows deployment


In [2]:
time = t.Time(c)
time.get_current_UTU()
time

< time at: 2023-07-11T06:52:28.763532+00:00 UTU:28298 >

In [3]:
time.get_global_actions()
time.actions

,action,job,agent
type,healthcare_initiatives,NaN,NaN
applies_to,pop,NaN,NaN
effort,2,NaN,NaN
requires_attr,{wealth: 0.1},NaN,NaN
augments_self_properties,"{health: 0.05, wealth: -.05}",NaN,NaN
comment,increase the health of your population slightl...,NaN,NaN
name,healthcare initiatives,takingAction,Shiyaos Hou
objid,8522332346729,NaN,3209702584262
username,Billmanserver,Billmanserver,Billmanserver
objtype,action,NaN,pop


All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


In [4]:
for i in time.actions.index:
    action = t.Action(c,time.actions.loc[i])
    print(action)
    

< (Shiyaos Hou:3209702584262) -takingAction-> (healthcare initiatives) >

In [ ]:
def resolve_augments_self_properties(agent, action):
    agent = agent.copy()
    self_properties = yaml.safe_load(action["augments_self_properties"])
    for p in self_properties.keys():
        agent[p] = agent[p] + float(self_properties[p])
    return agent


agent2 = resolve_augments_self_properties(agent, action)
results = pd.DataFrame([agent, agent2],).T
print("results of: ", action["type"])
results

NameError: name 'action' is not defined

In [ ]:
def query_patch_properties(agent, action):
    query = f"g.V().has('objid','{agent['objid']}')"
    for n in yaml.safe_load(action["augments_self_properties"]):
        query += f".property('{n}',{agent[n]})"
    return query


query_patch_properties(agent, action)

"g.V().has('objid','2570162977799').property('wealth',0.491).property('literacy',0.473).property('factionLoyalty',0.609)"

Veryify that actions are resolved or not

In [ ]:
actions_query = """
    g.V().haslabel('action').as('action')
                .inE('takingAction').has('status','resolved').as('job')
                .outV().as('agent')
                .select('action','job','agent')
"""

c.run_query(actions_query)
pd.DataFrame(c.res)

,action,job,agent
0,"{'id': '539f4d55-7dea-487a-8b62-756b0fe0d313',...","{'id': '6e029d81-83c9-4069-afa6-639aef48bd8f',...","{'id': 'eaa24c0e-8dcf-43d5-af89-ca6bb91d3111',..."
1,"{'id': '42011202-a68d-43d8-841f-87c50f42ac58',...","{'id': '5e9dce26-a02b-4972-a1c3-97d1f7d85f16',...","{'id': '083f7cac-dade-4961-b554-ad3009e704fa',..."
2,"{'id': 'c08bb02d-5739-49f4-a8c1-62c905f0bee5',...","{'id': 'e528b285-0e02-41a4-9226-0b94ffb8433c',...","{'id': '0f6f0e7b-4ce4-4390-a455-5f24a859a62d',..."
3,"{'id': 'aa2e3217-0a2b-47e9-abf7-c963ac28a543',...","{'id': 'e3fa5d2e-ce10-40dc-9367-9e95c3bba555',...","{'id': '90bcb328-de86-42d4-a59d-c182cc3eb995',..."
